In [1]:
!git clone https://github.com/Tmqng/ATR-Code

Cloning into 'ATR-Code'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (316/316), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 316 (delta 126), reused 272 (delta 89), pack-reused 0 (from 0)
Receiving objects: 100% (316/316), 6.93 MiB | 14.91 MiB/s, done.
Resolving deltas: 100% (126/126), done.


In [2]:
import os
os.getcwd()

import json
import subprocess
import zipfile

# Step 1: Set up Kaggle credentials
kaggle_username = "minhqunnguyen"
kaggle_api_key = "KGAT_874d32201f32459c120dc5947083a3d8"

os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)

kaggle_json = {
    "username": kaggle_username,
    "key": kaggle_api_key
}

json_path = os.path.expanduser('~/.kaggle/kaggle.json')
with open(json_path, 'w') as f:
    json.dump(kaggle_json, f)

os.chmod(json_path, 0o600)

# Step 2: Verify authentication works
result = subprocess.run(['kaggle', 'api', '-v'], capture_output=True, text=True)
print("Kaggle API version:", result.stdout)

# Step 3: Try downloading your private dataset
dataset_identifier = "minhqunnguyen/mstar-images-et-json"
output_dir = "/content/ATR-Code/datasets/MSTAR/"
os.makedirs(output_dir, exist_ok=True)

print(f"Attempting to download: {dataset_identifier}")
!cd {output_dir} && kaggle datasets download -d {dataset_identifier}

# Find the zip file
zip_file = os.path.join(output_dir, "mstar-images-et-json.zip")

if os.path.exists(zip_file):
    print(f"Extracting {zip_file}...")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    print("✓ Done!")
else:
    print(f"Zip file not found at {zip_file}")
    print(f"Files in {output_dir}:")
    for f in os.listdir(output_dir):
        print(f"  - {f}")

print("✓ Download complete!")

Kaggle API version: 
Attempting to download: minhqunnguyen/mstar-images-et-json
Dataset URL: https://www.kaggle.com/datasets/minhqunnguyen/mstar-images-et-json
License(s): unknown
 57% 155M/273M [00:00<00:00, 1.62GB/s]
100% 273M/273M [00:00<00:00, 968MB/s] 
Extracting /content/ATR-Code/datasets/MSTAR/mstar-images-et-json.zip...
✓ Done!
✓ Download complete!


In [3]:
from absl import logging
from absl import flags
from absl import app

from tqdm import tqdm

from torch.utils import tensorboard
from torch.utils.data import DataLoader, random_split

import torchvision
import torch

import numpy as np

import json

import sys
import os


import matplotlib.pyplot as plt
import numpy as np

# Get the project root
project_root = os.path.join(os.getcwd(), 'ATR-Code')

# Add src/ to path
sys.path.append(os.path.join(project_root, "src"))

# modules in src
from data.MSTAR.paper_AConvNet import preprocess
from data.MSTAR.paper_AConvNet import loader
from utils import common
from models import AConvNet

DATA_PATH = 'datasets/MSTAR/MSTAR_IMG_JSON'

# DATA_PATH = 'datasets/MSTAR/mstar_data_paper_AConvNet/'

model_str = 'AConvNet'


# flags.DEFINE_string('experiments_path', os.path.join(common.project_root, 'experiments'), help='')
# flags.DEFINE_string('config_name', f'{model_str}/config/AConvNet-SOC.json', help='')
# FLAGS = flags.FLAGS


common.set_random_seed(12321)

In [4]:
logging.info('Start')
# experiments_path = FLAGS.experiments_path
# config_name = FLAGS.config_name

# config = common.load_config(os.path.join(experiments_path, config_name))

experiments_path = os.path.join(common.project_root, 'experiments')

config = {
  "model_name": "AConvNet-SOC",
  "dataset": "SOC",
  "num_classes": 10,
  "channels": 1,
  "batch_size": 100,
  "epochs": 2,
  "momentum": 0.9,
  "lr": 1e-3,
  "lr_step": [50],
  "lr_decay": 0.1,
  "weight_decay": 4e-3,
  "dropout_rate": 0.5
}

dataset = config['dataset']
classes = config['num_classes']
channels = config['channels']
epochs = config['epochs']
batch_size = config['batch_size']

lr = config['lr']
lr_step = config['lr_step']
lr_decay = config['lr_decay']

weight_decay = config['weight_decay']
dropout_rate = config['dropout_rate']

model_name = config['model_name']

# run(epochs, dataset, classes, channels, batch_size,
#     lr, lr_step, lr_decay, weight_decay, dropout_rate,
#     model_name, experiments_path)

In [5]:
def load_dataset(path, is_train, name, batch_size):
    """
    Docstring for load_dataset

    :param path: Description
    :param is_train: Description
    :param name: Description
    :param batch_size: Description

    Load train, val or test dataset and apply transformations.
    """

    val_transform = torchvision.transforms.Compose([preprocess.CenterCrop(94)])

    train_transform = torchvision.transforms.Compose([preprocess.RandomCrop(94)])

    _dataset = loader.Dataset(
        path, name=name, is_train=is_train,
        transform=None
    )

    if is_train:

        # TODO Data_augmentation (in preprocess file)
        print(f"Augmenting training data with patches...")
        # Extract patches from training data
        augmented_samples = preprocess.augment_dataset_with_patches(
            _dataset,
            # patch_size=patch_size,
            # stride=stride,
            # chip_size=chip_size,
            desc="Train augmentation"
        )

        print(f"\nRésultats augmentation :")
        print(f"  Train : {len(_dataset)} images → {len(augmented_samples)} patches")
        print(f"  Facteur : ~{len(augmented_samples) / len(_dataset):.0f}x (13x13 = 169 patches/image)")

        augmented_dataset = preprocess.AugmentedDataset(augmented_samples)

        # augmented_dataset = _dataset

        # Split into train (80%) and validation (20%)
        train_size = int(0.8 * len(augmented_dataset))
        val_size = len(augmented_dataset) - train_size

        train_dataset, val_dataset = random_split(augmented_dataset, [train_size, val_size])

        # CenterCrop for val and RandomCrop for train
        train_dataset_transformed = preprocess.TransformWrapper(train_dataset, train_transform)
        val_dataset_transformed = preprocess.TransformWrapper(val_dataset, val_transform)

        train_data_loader = torch.utils.data.DataLoader(
            train_dataset_transformed, batch_size=batch_size, shuffle=is_train, num_workers=1
        )

        val_data_loader = torch.utils.data.DataLoader(
            val_dataset_transformed, batch_size=batch_size, shuffle=False, num_workers=1
        )

        return train_data_loader, val_data_loader


    else:
        test_dataset_transformed = preprocess.TransformWrapper(_dataset, val_transform)
        data_loader = torch.utils.data.DataLoader(
            test_dataset_transformed, batch_size=batch_size, shuffle=is_train, num_workers=1
        )
        return data_loader


@torch.no_grad()
def validation(m, ds):
    num_data = 0
    corrects = 0

    # Test loop
    m.net.eval()
    _softmax = torch.nn.Softmax(dim=1)
    for i, data in enumerate(tqdm(ds)):
        if i == 0:
            print(f"Data structure: {type(data)}")
            print(f"Data length: {len(data)}")
            images, labels, _ = data
            print(f"Images shape: {images.shape}")
            print(f"Labels shape: {labels.shape}")
            print(f"Unique labels in batch: {torch.unique(labels)}")
        images, labels, _ = data

        images = images.to(m.device)
        labels = labels.to(m.device)

        predictions = m.inference(images)
        predictions = predictions.to(m.device)
        predictions = _softmax(predictions)

        _, predictions = torch.max(predictions.data, 1)

        # DEBUG: Check predictions
        if i == 0:
            print(f"Predicted classes: {predictions[:10]}")
            print(f"True labels: {labels[:10]}")
            print(f"Matches: {(predictions == labels)[:10]}")

        labels = labels.type(torch.LongTensor)
        num_data += labels.size(0)
        corrects += (predictions == labels.to(m.device)).sum().item()

    accuracy = 100 * corrects / num_data
    return accuracy

In [6]:
# Décomposition de la fonction run

# Load data
train_set, val_set = load_dataset(DATA_PATH, True, dataset, batch_size)
test_set = load_dataset(DATA_PATH, False, dataset, batch_size)

load train data set: 100%|██████████| 2747/2747 [00:00<00:00, 3169.21it/s]


Augmenting training data with patches...


Train augmentation: 100%|██████████| 2747/2747 [00:08<00:00, 340.07it/s]



Résultats augmentation :
  Train : 2747 images → 134603 patches
  Facteur : ~49x (13x13 = 169 patches/image)


load test data set: 100%|██████████| 2425/2425 [00:00<00:00, 3111.45it/s]


In [7]:
import json
import os
from collections import defaultdict

path = os.path.join(project_root, "datasets/MSTAR/MSTAR_IMG_JSON/SOC")

# Dictionary to store class_ids by folder
train_classes = defaultdict(set)
test_classes = defaultdict(set)

# Check train folder
train_path = os.path.join(path, 'train')
for class_folder in os.listdir(train_path):
    class_folder_path = os.path.join(train_path, class_folder)
    if os.path.isdir(class_folder_path):
        for file in os.listdir(class_folder_path):
            if file.endswith('.json'):
                json_path = os.path.join(class_folder_path, file)
                with open(json_path, 'r') as f:
                    data = json.load(f)
                    class_id = data.get('class_id')
                    train_classes[class_folder].add(class_id)

# Check test folder
test_path = os.path.join(path, 'test')
for class_folder in os.listdir(test_path):
    class_folder_path = os.path.join(test_path, class_folder)
    if os.path.isdir(class_folder_path):
        for file in os.listdir(class_folder_path):
            if file.endswith('.json'):
                json_path = os.path.join(class_folder_path, file)
                with open(json_path, 'r') as f:
                    data = json.load(f)
                    class_id = data.get('class_id')
                    test_classes[class_folder].add(class_id)

# Print results
print("="*60)
print("TRAIN SET - class_id values by folder:")
print("="*60)
for folder in sorted(train_classes.keys()):
    print(f"{folder:20s}: {train_classes[folder]}")

print("\n" + "="*60)
print("TEST SET - class_id values by folder:")
print("="*60)
for folder in sorted(test_classes.keys()):
    print(f"{folder:20s}: {test_classes[folder]}")

print("\n" + "="*60)
print("SUMMARY:")
print("="*60)
print(f"Train folders: {len(train_classes)}")
print(f"Test folders: {len(test_classes)}")


TRAIN SET - class_id values by folder:
2S1                 : {0}
BMP2                : {1}
BRDM2               : {2}
BTR60               : {3}
BTR70               : {4}
D7                  : {5}
T62                 : {6}
T72                 : {7}
ZIL131              : {8}
ZSU234              : {9}

TEST SET - class_id values by folder:
2S1                 : {0}
BMP2                : {1}
BRDM2               : {2}
BTR60               : {3}
BTR70               : {4}
D7                  : {5}
T62                 : {6}
T72                 : {7}
ZIL131              : {8}
ZSU234              : {9}

SUMMARY:
Train folders: 10
Test folders: 10


In [8]:
m = AConvNet.Model(
    classes=classes, dropout_rate=dropout_rate, channels=channels,
    lr=lr, lr_step=lr_step, lr_decay=lr_decay,
    weight_decay=weight_decay
)

model_path = os.path.join(experiments_path, f'{model_str}/models/{model_name}')
if not os.path.exists(model_path):
    os.makedirs(model_path, exist_ok=True)

history_path = os.path.join(experiments_path, f'{model_str}/history')
if not os.path.exists(history_path):
    os.makedirs(history_path, exist_ok=True)

history = {
    'train_loss': [],
    'train_accuracy': [],
    'val_loss': [],
    'val_accuracy': []
}

for epoch in range(epochs):
    _loss = []

    m.net.train()
    for i, data in enumerate(tqdm(train_set)):
        images, labels, _ = data
        _loss.append(m.optimize(images, labels))

    if m.lr_scheduler:
        lr = m.lr_scheduler.get_last_lr()[0]
        m.lr_scheduler.step()

    train_accuracy = validation(m, train_set)
    val_accuracy = validation(m, val_set)

    logging.info(
        f'Epoch: {epoch + 1:03d}/{epochs:03d} | loss={np.mean(_loss):.4f} | lr={lr} | Train accuracy={train_accuracy:.2f} | Validation accuracy={val_accuracy:.2f}'
    )

    history['train_loss'].append(np.mean(_loss))
    history['train_accuracy'].append(train_accuracy)
    history['val_accuracy'].append(val_accuracy)

    if experiments_path:
        m.save(os.path.join(model_path, f'model-{epoch + 1:03d}.pth'))

    with open(os.path.join(history_path, f'history-{model_name}.json'), mode='w', encoding='utf-8') as f:
        json.dump(history, f, ensure_ascii=True, indent=2)

Device used: cuda


  0%|          | 1/1077 [00:00<03:59,  4.50it/s]

Data structure: <class 'list'>
Data length: 3
Images shape: torch.Size([100, 1, 94, 94])
Labels shape: torch.Size([100])
Unique labels in batch: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Predicted classes: tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6], device='cuda:0')
True labels: tensor([4, 2, 7, 6, 4, 2, 8, 6, 6, 6], device='cuda:0')
Matches: tensor([False, False, False,  True, False, False, False,  True,  True,  True],
       device='cuda:0')


  2%|▏         | 5/270 [00:00<00:10, 24.68it/s]

Data structure: <class 'list'>
Data length: 3
Images shape: torch.Size([100, 1, 94, 94])
Labels shape: torch.Size([100])
Unique labels in batch: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Predicted classes: tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6], device='cuda:0')
True labels: tensor([5, 0, 4, 5, 4, 7, 0, 6, 2, 7], device='cuda:0')
Matches: tensor([False, False, False, False, False, False, False,  True, False, False],
       device='cuda:0')


  0%|          | 5/1077 [00:00<00:42, 25.05it/s]

Data structure: <class 'list'>
Data length: 3
Images shape: torch.Size([100, 1, 94, 94])
Labels shape: torch.Size([100])
Unique labels in batch: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Predicted classes: tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6], device='cuda:0')
True labels: tensor([5, 0, 0, 0, 0, 6, 5, 2, 7, 1], device='cuda:0')
Matches: tensor([False, False, False, False, False,  True, False, False, False, False],
       device='cuda:0')


  2%|▏         | 5/270 [00:00<00:11, 24.00it/s]

Data structure: <class 'list'>
Data length: 3
Images shape: torch.Size([100, 1, 94, 94])
Labels shape: torch.Size([100])
Unique labels in batch: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Predicted classes: tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6], device='cuda:0')
True labels: tensor([5, 0, 4, 5, 4, 7, 0, 6, 2, 7], device='cuda:0')
Matches: tensor([False, False, False, False, False, False, False,  True, False, False],
       device='cuda:0')


100%|██████████| 270/270 [00:04<00:00, 59.02it/s]
